# Create dataset using artificial skull models

This notebook runs wavefield simulations containing artificial skull models, and captures images of the borders of the wavefield to create a dataset.

In [ ]:
!pip install git+https://github.com/devitocodes/devito.git --quiet
!pip install sympy==1.7.0 --quiet
!pip install numpy==1.17.0 --quiet

     |████████████████████████████████| 81kB 2.2MB/s 
     |████████████████████████████████| 102kB 6.3MB/s 
     |████████████████████████████████| 143kB 18.8MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 194kB 36.1MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 71kB 9.2MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
ERROR: pytest-cov 2.12.1 has requirement coverage>=5.2.1, but you'll have coverage 3.7.1 which is incompatible.
ERROR: pytest-cov 2.12.1 has requirement pytest>=4.6, but you'll have pytest 3.6.4 which is incompatible.
     |████████████████████████████████| 5.9MB 2.9MB/s 
     |████████████████████████████████| 20.3MB 1.4MB/s 
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.17.0 which is incompatible.
ERROR: kapre 0.3.5 has requirement numpy>=1.18.5, but you'll have numpy 1.17.0 which is incompatible.
ERROR: datascience 0.10.6

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/My\ Drive/ML_Ultrasound_Project/UROPdata

/content/drive/My Drive/ML_Ultrasound_Project/UROPdata


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy

import devito
from sympy import Symbol
from devito import Operator
from devito import Eq, solve
from devito import TimeFunction

from medical.plotting import plot_shotrecord, plot_velocity
from medical.source import ToneBurstSource, Receiver
from medical.source import TimeAxis
from medical.model import Model

from tqdm.notebook import tqdm

from mpl_toolkits.axes_grid1 import make_axes_locatable

# devito.parameters.configuration['dse'] = 'advanced'
# devito.parameters.configuration['dle'] = 'speculative'
devito.parameters.configuration['openmp'] = True
devito.parameters.configuration['mpi'] = False

def create_eliptic_array(n_sources, boundary_offset, dx, shape):
    a = (shape[1] / 2)
    b = (shape[0] / 2)

    pos = np.empty((2, n_sources), dtype=np.float32)
    angle = np.zeros((n_sources))
    for idx in range(n_sources):
        angle[idx] = ((idx) * 360.0 / n_sources) * 2 * np.pi / 360.0  # transducer angle
        pos[1, idx] = (a * np.cos(angle[idx]) + shape[1] / 2)  # spherical equation of ellipse
        pos[0, idx] = (b * np.sin(angle[idx]) + shape[0] / 2)  # spherical equation of ellipse

    pos[0, :] += boundary_offset[0]
    pos[1, :] += boundary_offset[1]
    pos *= dx
    # np.array((np.divide(grid[0] ** 2, major_ax ** 2) + np.divide(grid[1] ** 2, minor_ax ** 2)) < 1, dtype=np.float32)
    return pos

Trying to access deprecated config `openmp`. Using `language` instead


**Functions for creating samples**

In [ ]:
def get_images(basedata, t, grid_shape):

  # Here we get 9 images of the wavefield from the past, 6 timesteps apart,
  # i.e. t, t-6, t-12, ..., t-48.
  
  if t < 6:
    image0 = np.zeros(grid_shape)
    image1 = np.zeros(grid_shape)
    image2 = np.zeros(grid_shape)
    image3 = np.zeros(grid_shape)
    image4 = np.zeros(grid_shape)
    image5 = np.zeros(grid_shape)
    image6 = np.zeros(grid_shape)
    image7 = np.zeros(grid_shape)
    image8 = basedata[t]
    
  elif t < 12:
    image0 = np.zeros(grid_shape)
    image1 = np.zeros(grid_shape)
    image2 = np.zeros(grid_shape)
    image3 = np.zeros(grid_shape)
    image4 = np.zeros(grid_shape)
    image5 = np.zeros(grid_shape)
    image6 = np.zeros(grid_shape)
    image7 = basedata[t-6]
    image8 = basedata[t]    

  elif t < 18:
    image0 = np.zeros(grid_shape)
    image1 = np.zeros(grid_shape)
    image2 = np.zeros(grid_shape)
    image3 = np.zeros(grid_shape)
    image4 = np.zeros(grid_shape)
    image5 = np.zeros(grid_shape)
    image6 = basedata[t-12]
    image7 = basedata[t-6]
    image8 = basedata[t]

  elif t < 24:
    image0 = np.zeros(grid_shape)
    image1 = np.zeros(grid_shape)
    image2 = np.zeros(grid_shape)
    image3 = np.zeros(grid_shape)
    image4 = np.zeros(grid_shape)
    image5 = basedata[t-18]
    image6 = basedata[t-12]
    image7 = basedata[t-6]
    image8 = basedata[t]

  elif t < 30:
    image0 = np.zeros(grid_shape)
    image1 = np.zeros(grid_shape)
    image2 = np.zeros(grid_shape)
    image3 = np.zeros(grid_shape)
    image4 = basedata[t-24]
    image5 = basedata[t-18]
    image6 = basedata[t-12]
    image7 = basedata[t-6]
    image8 = basedata[t]

  elif t < 36:
    image0 = np.zeros(grid_shape)
    image1 = np.zeros(grid_shape)
    image2 = np.zeros(grid_shape)
    image3 = basedata[t-30]
    image4 = basedata[t-24]
    image5 = basedata[t-18]
    image6 = basedata[t-12]
    image7 = basedata[t-6]
    image8 = basedata[t]

  elif t < 42:
    image0 = np.zeros(grid_shape)
    image1 = np.zeros(grid_shape)
    image2 = basedata[t-36]
    image3 = basedata[t-30]
    image4 = basedata[t-24]
    image5 = basedata[t-18]
    image6 = basedata[t-12]
    image7 = basedata[t-6]
    image8 = basedata[t]

  elif t < 48:
    image0 = np.zeros(grid_shape)
    image1 = basedata[t-42]
    image2 = basedata[t-36]
    image3 = basedata[t-30]
    image4 = basedata[t-24]
    image5 = basedata[t-18]
    image6 = basedata[t-12]
    image7 = basedata[t-6]
    image8 = basedata[t]

  else:
    image0 = basedata[t-48]
    image1 = basedata[t-42]
    image2 = basedata[t-36]
    image3 = basedata[t-30]
    image4 = basedata[t-24]
    image5 = basedata[t-18]
    image6 = basedata[t-12]
    image7 = basedata[t-6]
    image8 = basedata[t]

  return image0, image1, image2, image3, image4, image5, image6, image7, image8

In [ ]:
def create_samples(image0, image1, image2, image3, image4, image5, image6, image7, image8, i, slices, rot_num):

  # This function creates network input images and targets from wavefield borders

  # Create square 23x23 network input images
  roi0 = np.array(np.rot90(image0[slices],rot_num))
  square_timestep0 = roi0[i:i+23, 11:34]
  roi1 = np.array(np.rot90(image1[slices],rot_num))
  square_timestep1 = roi1[i:i+23, 11:34]    
  roi2 = np.array(np.rot90(image2[slices],rot_num))
  square_timestep2 = roi2[i:i+23, 11:34]
  roi3 = np.array(np.rot90(image3[slices],rot_num))
  square_timestep3 = roi3[i:i+23, 11:34]
  roi4 = np.array(np.rot90(image4[slices],rot_num))
  square_timestep4 = roi4[i:i+23, 11:34]
  roi5 = np.array(np.rot90(image5[slices],rot_num))
  square_timestep5 = roi5[i:i+23, 11:34]
  roi6 = np.array(np.rot90(image6[slices],rot_num))
  square_timestep6 = roi6[i:i+23, 11:34]
  roi7 = np.array(np.rot90(image7[slices],rot_num))
  square_timestep7 = roi7[i:i+23, 11:34]
  roi8 = np.array(np.rot90(image8[slices],rot_num))
  square_timestep8 = roi8[i:i+23, 11:34]

  #link the 9 images together
  newdata = np.vstack((square_timestep0, square_timestep1))
  newdata = np.vstack((newdata, square_timestep2))
  newdata = np.vstack((newdata, square_timestep3))
  newdata = np.vstack((newdata, square_timestep4))
  newdata = np.vstack((newdata, square_timestep5))
  newdata = np.vstack((newdata, square_timestep6))
  newdata = np.vstack((newdata, square_timestep7))
  newdata = np.vstack((newdata, square_timestep8))

  newdata = np.reshape(newdata, (9,23,23))

  # Create corresponding labels
  label = roi8[i+11:i+12, 0:11]
  label_T = np.transpose(label)

  return newdata, label_T


In [ ]:
skulls = np.load('/content/drive/My Drive/ML_Ultrasound_Project/UROPdata/data/fakeSkulls.npy')

y_length = 480+66+4  #480+11
x_length = 350+66+4

num = 2500              # Simulation last number of time steps - CHANGE THIS FOR THE AMOUNT OF TIMESTEPS

shape = (1001, 1001) #(1001, 1001)            # Number of grid point (nx, nz) - CHANGE THIS FOR THE MEDIUM SIZE
size_diff = int((shape[0]-1001)/2)

top = (slice(225-1-11+size_diff, -226-11+11-size_diff), slice(290+size_diff, 290+34+size_diff))
right = (slice(-226-34-size_diff, -226-size_diff), slice(290+19-11+size_diff, -291-11+11-size_diff))
bottom = (slice(225-1-11+size_diff, -226-11+11-size_diff), slice(-291-34-size_diff, -291-size_diff))
left = (slice(225+size_diff, 225+34+size_diff), slice(290+19-11+size_diff, -291-11+11-size_diff))

area = (slice(225+size_diff, -226-size_diff), slice(290+size_diff, -291-size_diff))

skull_start = 0
for n, m in tqdm(enumerate(range(skull_start, len(skulls)))):
  n+=skull_start
  print(n)
  model = skulls[m]
  src_index = np.random.randint(512)  # Choose random point source location

  # Define a physical size
  spacing = (0.5e-3, 0.5e-3)      # Grid spacing in m.
  origin = (0., 0.)               # What is the location of the top left corner. This is necessary to define
                                # the absolute location of the source and receivers

  # Define a velocity profile. The velocity is in m/s
  diff_x = shape[0] - model.shape[0]
  diff_y = shape[1] - model.shape[1]
  offset_x = int(np.floor(diff_x/2))
  offset_y = int(np.floor(diff_y/2))

  model = np.pad(model,
              ((int(np.floor(diff_x/2)), int(np.ceil(diff_x/2))), (int(np.floor(diff_y/2)), int(np.ceil(diff_y/2)))),
              'edge')

  plt.imshow(np.transpose(model), origin='lower')

  # With the velocity and model size defined, we can create the seismic model that
  # encapsulates this properties. We also define the size of the absorbing layer as 10 grid points
  model = Model(vp=model, origin=origin, shape=shape, spacing=spacing, space_order=4, nbpml=0)

  t0 = 0.                 # Simulation starts a t=0
  dt = 0.08e-6            # Time step from model grid spacing

  time_range = TimeAxis(start=t0, num=num, step=dt)

  f0 = 0.40e6  # Source peak frequency is 400kHz
  src = ToneBurstSource(name='src', grid=model.grid, f0=f0, npoint=1, time_range=time_range)

  # First, position source centrally in all dimensions
  src_positions = create_eliptic_array(512, (offset_x, offset_y), spacing[0], (480, 350))
  src.coordinates.data[0, :] = src_positions[:, src_index]

  # plt.scatter(src_positions[0, :]/spacing[0], src_positions[1, :]/spacing[1])
  # plt.show()

  # # We can plot the time signature to see the wavelet
  # src.show()

  # We can now show the source and receivers within our domain:
  # Red dot: Source location
  plot_velocity(model, source=src.coordinates.data)

  # In order to represent the wavefield u and the square slowness we need symbolic objects
  # corresponding to time-space-varying field (u, TimeFunction) and
  # space-varying field (m, Function)

  # Define the wavefield with the size of the model and the time dimension
  u = TimeFunction(name="u", grid=model.grid, time_order=2, space_order=4, save=num)

  # Create a temporary symbol for H to avoid expensive sympy solve
  H = Symbol('H')

  # Define PDE
  eq = model.m * u.dt2 - H

  # Solve the symbolic equation for the field to be updated
  eq_time = solve(eq, u.forward)

  # Get the spacial FD
  biharmonic = u.biharmonic(1/model.m)
  laplacian = u.laplace + dt**2/12 * biharmonic

  # This discrete PDE can be solved in a time-marching way updating u(t+dt) from the previous time step
  # Devito as a shortcut for u(t+dt) which is u.forward. We can then rewrite the PDE as
  # a time marching updating equation known as a stencil using customized SymPy functions
  stencil = Eq(u.forward, eq_time.subs({H: laplacian}))

  # Finally we define the source injection function to generate the corresponding code
  src_term = src.inject(field=u.forward, expr=src * dt**2 / model.m)

  # Execute the operator
  u.data.fill(0.)
  op = Operator([stencil] + src_term, subs=model.spacing_map)
  op(dt=dt)  # AFTER THIS LINE u.data HAS THE WAVEFIELD AT ALL TIMESTEPS WITH SHAPE (num, nx, ny)

  skull_images = []
  skull_labels = []

  for t in tqdm(range(0, num, 20)):
    if np.any(u.data[t][top] != 0):
      image0, image1, image2, image3, image4, image5, image6, image7, image8 = get_images(u.data, t, shape)
      for i in range(540):
        ts_images, ts_label = create_samples(image0, image1, image2, image3, image4, image5, image6, image7, image8, i, top, 0)
        skull_images.append(ts_images)
        skull_labels.append(ts_label)
        if (i+1)%30 == 0:
          if np.all(np.abs(np.array(skull_images[-30:])) < 0.5e-9):
            skull_images = skull_images[:-30]
            skull_labels = skull_labels[:-30]
          else:
            print('top')
            plt.figure()
            plt.imshow(skull_images[-12][0,:,:])
            plt.show()
            plt.figure()
            plt.imshow(skull_images[-12][8,:,:])
            plt.show()
          #   plt.figure()
          #   plt.imshow(np.hstack((np.squeeze(skull_labels[-23:]), skull_images[-12][8,:,:])))
          #   plt.colorbar()
          #   plt.show()

    if np.any(u.data[t][right] != 0):
      image0, image1, image2, image3, image4, image5, image6, image7, image8 = get_images(u.data, t, shape)
      for i in range(390):
        ts_images, ts_label = create_samples(image0, image1, image2, image3, image4, image5, image6, image7, image8, i, right, 3)
        skull_images.append(ts_images)
        skull_labels.append(ts_label)
        if (i+1)%30 == 0:
          if np.all(np.abs(np.array(skull_images[-30:])) < 0.5e-9):
            skull_images = skull_images[:-30]
            skull_labels = skull_labels[:-30]
          # else:
          #   print('right')
          #   plt.figure()
          #   plt.imshow(np.hstack((np.squeeze(skull_labels[-23:]), skull_images[-12][8,:,:])))
          #   plt.colorbar()
          #   plt.show()

    if np.any(u.data[t][bottom] != 0):
      image0, image1, image2, image3, image4, image5, image6, image7, image8 = get_images(u.data, t, shape)
      for i in range(540):
        ts_images, ts_label = create_samples(image0, image1, image2, image3, image4, image5, image6, image7, image8, i, bottom, 2)
        skull_images.append(ts_images)
        skull_labels.append(ts_label)
        if (i+1)%30 == 0:
          if np.all(np.abs(np.array(skull_images[-30:])) < 0.5e-9):
            skull_images = skull_images[:-30]
            skull_labels = skull_labels[:-30]
          # else:
          #   print('bottom')
          #   plt.figure()
          #   plt.imshow(np.hstack((np.squeeze(skull_labels[-23:]), skull_images[-12][8,:,:])))
          #   plt.colorbar()
          #   plt.show()

    if np.any(u.data[t][left] != 0):
      image0, image1, image2, image3, image4, image5, image6, image7, image8 = get_images(u.data, t, shape)
      for i in range(390):
        ts_images, ts_label = create_samples(image0, image1, image2, image3, image4, image5, image6, image7, image8, i, left, 1)
        skull_images.append(ts_images)
        skull_labels.append(ts_label)
        if (i+1)%30 == 0:
          if np.all(np.abs(np.array(skull_images[-30:])) < 0.5e-9):
            skull_images = skull_images[:-30]
            skull_labels = skull_labels[:-30]
          # else:
          #   print('left')
          #   plt.figure()
          #   plt.imshow(np.hstack((np.squeeze(skull_labels[-23:]), skull_images[-12][8,:,:])))
          #   plt.colorbar()
          #   plt.show()

  skull_images = np.array(skull_images)
  skull_labels = np.array(skull_labels)

  np.savez_compressed('/content/drive/My Drive/ML_Ultrasound_Project/UROPdata/skull_images_{}'.format(n), a=skull_images)
  np.savez_compressed('/content/drive/My Drive/ML_Ultrasound_Project/UROPdata/skull_labels_{}'.format(n), a=skull_labels)

  print('{} samples'.format(len(skull_labels)))